# データ読込

In [1]:
import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite
from utils.get_holydays import get_japanese_holidays
from logic.factory_manage.predict_model_v3 import generate_features, train_and_predict

# CSVファイル読み込み（パスは適宜変更）
path = "/work/app/data/factory_manage/weight_data.db"
df = load_data_from_sqlite()
df["伝票日付"].max()

hol_max = df["伝票日付"].max()
hol_min = df["伝票日付"].min()
print(f"最小日付: {hol_min}, 最大日付: {hol_max}")
holiday = get_japanese_holidays(hol_min, hol_max)


最小日付: 2024-05-01 00:00:00, 最大日付: 2025-06-22 00:00:00


In [2]:
df_reserve = pd.read_csv(
    "/work/app/data/input/yoyaku_data.csv")
df_reserve["予約日"] = pd.to_datetime(df_reserve["予約日"])
df_reserve = df_reserve[df_reserve["予約日"] >= hol_min]
df_reserve = df_reserve[df_reserve["予約日"] <= hol_max]
df_reserve

,予約日,予約得意先名,固定客
0,2025-03-02,泉土木,True
1,2025-03-02,ASMU,False
2,2025-03-02,青木サービス,False
3,2025-03-02,オネストワン,False
4,2025-03-02,おかたずけ田中くん,False
...,...,...,...
4743,2025-05-31,山口興業,True
4744,2025-05-31,谷津商会,True
4745,2025-05-31,吉田機電,False
4746,2025-05-31,ヨシモリ,True


In [3]:
print(df_reserve.columns)

Index(['予約日', '予約得意先名', '固定客'], dtype='object')


## 特徴量生成

In [4]:
from logic.factory_manage.predict_model_v4_3 import full_walkforward_pipeline

pred_df = full_walkforward_pipeline(
    df,
    holidays=holiday,
    df_reserve=df_reserve
)



===== 2024-05-10 処理中 =====
    混合廃棄物A 予測値: 51000.0kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 1.0083
rf: 0.0000
    混合廃棄物B 予測値: 5320.0kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 1.0100
rf: 0.0000

===== 2024-05-11 処理中 =====
    混合廃棄物A 予測値: 45616.7kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 1.0126
rf: 0.0001
    混合廃棄物B 予測値: 9259.2kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 1.0854
rf: -0.0796

===== 2024-05-12 処理中 =====
    混合廃棄物A 予測値: 42995.9kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 0.9347
rf: 0.1454
    混合廃棄物B 予測値: 14870.6kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 1.3126
rf: -0.4332

===== 2024-05-13 処理中 =====
    混合廃棄物A 予測値: 9913.3kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 1.1562
rf: -0.1895
    混合廃棄物B 予測値: -84.0kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: -0.0370
rf: 2.3302

===== 2024-05-14 処理中 =====
    混合廃棄物A 予測値: 43513.9kg

===== ステージ1メタモデル係数 (ElasticNet) =====
elastic: 0.6161
rf: 0.9394
    混合廃棄物B 予測値: 12564.4kg

===== 